<h1>Growth Regularisation</h1>
This notebook demonstrates the implementation of this paper <a href=https://arxiv.org/abs/2012.09243> Neural Pruning Via Growth Regularisation</a>
<h4>Steps to train a baseline model and then compress it given a channel budget are as follows:</h4>
<ul>
    <li>Load the YAML file. </li>
    <li>Load dataset and create dataloaders. </li>
    <li>Create <b>Growth Regularisation</b> object and pass the parameters in the form of a dictionary. </li>
    <li>Pass the dataloaders into the <b>compress_model</b> method to obtain the compressed model. </li>
</ul>
Since this is a demo notebook the number of epochs have been set to 2.

In [1]:
import sys
import os

sys.path.append("../../../")
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import torch
from torchvision import transforms

import yaml

from trailmet.datasets.classification import DatasetFactory
from trailmet.models import ModelsFactory
from trailmet.algorithms.prune.growth_regularisation import Growth_Regularisation

/opt/conda/envs/py117/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root = "./"

<h3> Loading the YAML file. </h3>

In [3]:
with open(os.path.join(root, "resnet50_cifar100.yaml"), "r") as stream:
    data_loaded = yaml.safe_load(stream)
data_loaded

{'num_classes': 100,
 'weight_decay': 0.0005,
 'arch': 'resnet50',
 'dataset': 'cifar100',
 'epochs': 1,
 'learning_rate': 0.002,
 'prune_ratio': 0.5,
 'reg_upper_limit': 0.0004,
 'reg_granularity_prune': 0.0001,
 'method': 'GReg-1',
 'wandb': True,
 'base_pr_model': False,
 'stage_pr': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
 'skip_layers': '',
 'inherit_pruned': 'index',
 'pick_pruned': 'min',
 'copy_bn_w': True,
 'copy_bn_b': True,
 'wg': 'filter',
 'print_interval': 100,
 'lr_prune': 0.001,
 'momentum': 0.9,
 'lr_pick': 0.001,
 'resume_path': False,
 'update_reg_interval': 5,
 'stabilize_reg_interval': 40000,
 'mag_ratio_limit': 1000,
 'reg_granularity_recover': 0.0001,
 'plot_interval': 5,
 'save_order_log': False,
 'save_mag_reg_log': False,
 'save_interval': 5,
 'block_loss_grad': False,
 'insize': 32}

<h3>Loading CIFAR100Dataset</h3>

In [4]:
transform_train = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Pad(4, padding_mode="reflect"),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomCrop(32),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ]
)

transform_test = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ]
)

transforms1 = {"train": transform_train, "val": transform_test, "test": transform_test}

target_transforms = {"train": None, "val": None, "test": None}

cifar_dataset = DatasetFactory.create_dataset(
    name="CIFAR100",
    root="./data",
    split_types=["train", "val", "test"],
    val_fraction=0.1,
    transform=transforms1,
    target_transform=target_transforms,
    random_seed=42,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


<h5>Creating the dataloaders</h5>

In [5]:
dataloaders = {
    "train": torch.utils.data.DataLoader(
        cifar_dataset["train"],
        batch_size=64,
        sampler=cifar_dataset["train_sampler"],
        num_workers=0,
    ),
    "val": torch.utils.data.DataLoader(
        cifar_dataset["val"],
        batch_size=64,
        sampler=cifar_dataset["val_sampler"],
        num_workers=0,
    ),
    "test": torch.utils.data.DataLoader(
        cifar_dataset["test"],
        batch_size=64,
        sampler=cifar_dataset["test_sampler"],
        num_workers=0,
    ),
}

<h3> Creating the method's object proceed with compression. </h3>

In [6]:
model = ModelsFactory.create_model(name="resnet50", pretrained=False, **data_loaded)

In [7]:
slim = Growth_Regularisation(model, dataloaders, **data_loaded)

wandb: Currently logged in as: animesh-007. Use `wandb login --relogin` to force relogin


In [ ]:
slim.compress_model()